### 투표 기반 분류기

앙상블 기법은 **예측기가 가능한 한 서로 독립적일 때 최고의 성능을 발휘**함 -> 서로 다른 알고리즘으로 학습시킬 경우 다양한 분류기를 얻을 수 있기 때문에 다른 종류의 오차를 만들 가능성이 높음

In [1]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 간접 투표 방식으로 바꿀 경우 voting='soft', SVC probability=True 지정
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)

In [2]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [7]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    # print(clf)
    clf.fit(X, y)
    
    y_pred = clf.predict(X_test)
    
    print(clf.__class__.__name__, '\t', accuracy_score(y_test, y_pred))

LogisticRegression 	 0.848
RandomForestClassifier 	 1.0
SVC 	 0.912
VotingClassifier 	 0.944


### 배깅과 페이스팅

배깅 : 훈련 세트에서 **중복을 허용하여** 샘플링하는 방식   
페이스팅 : **중복을 허용하지 않고** 샘플링하는 방식

In [9]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=1
)

In [10]:
bag_clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=1)

In [11]:
y_pred = bag_clf.predict(X_test)

In [12]:
print(y_pred)

[1 0 0 1 1 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 1 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0
 1 1 1 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1 1 0 1 0 1 1 0 1 0 0 0 0 1 0 0 1 1
 0 0 1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 1 0
 0 0 1 1 0 0 0 0 1 1 1 0 0 0]


### 랜덤 패치와 랜덤 서브스페이스

랜덤 패치 : 훈련 특성과 샘플을 모두 샘플링   
랜덤 서브스페이스 : 훈련 샘플은 모두 사용하고 특성은 샘플링   

### 랜덤 포레스트

In [14]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=1)
rnd_clf.fit(X_train, y_train)

y_pred = rnd_clf.predict(X_test)

In [15]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

### 부스팅 - AdaBoost

In [16]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5)

ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 부스팅 - 그래디언트 부스팅

In [18]:
import numpy as np

np.random.seed(42)

# 잡음이 섞인 2차 곡선 형태의 훈련 세트
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [19]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [21]:
# 첫 번째 예측기에서 생긴 잔여 오차에 두 번째 DecisionTreeRegressor 훈련
y2 = y-tree_reg1.predict(X)

tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

In [22]:
# 두 번째 예측기에서 생긴 잔여 오차에 세 번째 회귀 모델 훈련
y3 = y2-tree_reg2.predict(X)

tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [24]:
for tree in (tree_reg1, tree_reg2, tree_reg3):
    print(tree)
    
X_new = np.array([[0.8]])

y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

DecisionTreeRegressor(max_depth=2)
DecisionTreeRegressor(max_depth=2)
DecisionTreeRegressor(max_depth=2)
